### intial set-up

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sb
pd.set_option('display.max_columns', None) #all columns are displayed
pd.set_option('display.max_rows', None) #all rows are displayed

In [40]:
wine = pd.read_csv("wine_data_white.csv",sep=";")
wine['alcohol'] = pd.to_numeric(wine['alcohol'], errors = 'coerce')
wine = wine.dropna(subset=['alcohol'])
wine.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

### Data split & NeighborsClassifier

In [41]:
from sklearn.model_selection import train_test_split

xwine = wine.drop('quality', axis =1)
xtrn, xtst, ytrn, ytst = train_test_split(xwine, wine['quality'], random_state = 42, test_size = 0.2)


In [42]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(xtrn, ytrn)
y_pred = knn.predict(xtst)

In [80]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
root_mean_squared_error (ytst, y_pred)
mean_absolute_error(ytst, y_pred)

#knn.score(xtst, ytst) #accuracy

0.42754367934224047

### note 
would be nice to plot the knn prob results and the distance towards the actuals. mlxtend is not supported by anaconda, asked Ilias


In [44]:
#import mlxtend.plotting as mlx

#from mlxtend.plotting import plot_decision_regions 
#plot_decision_regions(X, y, clf=knn, legend=2)  

#plt.xlabel('X')  
#plt.ylabel('Y')
#plt.title('KNN with K=5')

#plt.savefig('KNN with K=5.jpeg', bbox_inches="tight", dpi=150)

#plt.show()

In [45]:
#import statsmodels.api as sm

#df["predicted_write"] = result.predict(df)

#sns.scatterplot(x="write", y="predicted_write", data=df)
#plt.xlabel("Actual Writing Score")
#plt.ylabel("Predicted Writing Score")
#plt.title("Actual vs. Predicted Writing Scores")
#plt.show()b

### Pipeline & cross validation

In [81]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
pipe = make_pipeline(scaler, knn)
pipe.fit(xtrn, ytrn)
y_pred = pipe.predict(xtst)
root_mean_squared_error(y_pred, ytst)
mean_absolute_error(y_pred, ytst)

0.42754367934224047

Note: check monday, scaling minimal adjusted performance metric. Would be nice to plot this

In [ ]:
from sklearn.model_selection import cross_val_score
knn_mse = -cross_val_score(pipe, xtrn, ytrn, scoring = 'neg_mean_squared_error', cv = 5)

0    0.685494
1    0.614396
2    0.650386
3    0.667095
4    0.616967
dtype: float64

In [93]:
np.mean(knn_mse)
np.max(knn_mse) - np.min(knn_mse) 
np.std(knn_mse)

0.027792185473292246

In [103]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
y_predict = cross_val_predict(pipe,xtrn, ytrn, cv =5)
cm = confusion_matrix(ytrn, y_predict)
print(cm)

[[   0    2    6    5    0    1    0]
 [   1   40   55   35    5    2    0]
 [   2   48  739  331   42    3    0]
 [   3   21  357 1119  213   36    0]
 [   0    5   45  211  364   52    1]
 [   0    0    6   29   42   65    0]
 [   0    0    1    1    3    0    0]]


??